## 1. Preparación del entorno

In [29]:
# Instalación de librerías
!pip install gradio joblib xgboost scikit-learn
import gradio as gr
import pandas as pd
import joblib

In [30]:
# Conexión con Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Cargar el modelo XGBoost

In [31]:
# Carga del modelo
file_path = '/content/drive/MyDrive/mejor_modelo_xgboost.pkl'
modelo_xgb = joblib.load(file_path)

# Comprobación de modelo con pipeline creado
print(type(modelo_xgb))

<class 'sklearn.pipeline.Pipeline'>


## 3. Desarrollo de la aplicación

#### 3.1. Diccionario de mapeos

In [32]:
# Mapeos originales del modelo
age_map = {
    '<15 años': '<15', '15-19 años': '15-19', '20-24 años': '20-24',
    '25-29 años': '25-29', '30-34 años': '30-34', '35-39 años': '35-39',
    '40-44 años': '40-44', '45-49 años': '45-49', '50-54 años': '50-54'
}
race_map = {
    'Blanca': 'White', 'Negra': 'Black',
    'Indígena Americano/Nativo de Alaska': 'AIAN',
    'Asiática': 'Asian', 'Isleña del Pacífico': 'NHOPI',
    'Multiracial': 'Multiracial'
}
edu_map = {
    'Educación Primaria': '≤8th',
    'Educación Secundaria': '9-12_no_diploma',
    'Bachillerato': 'HS_diploma',
    'Estudios Pre-Universitarios': 'Some_college',
    'Grado Superior': 'Assoc_degree',
    'Graduado Universitario': 'Bachelor',
    'Máster Universitario': 'Master',
    'Doctorado': 'Doctorate',
    'Desconocido': 'Unknown'
}
bmi_map = {
    'Infrapeso': 'Underweight',
    'Normal': 'Normal',
    'Sobrepeso': 'Overweight',
    'Obesidad I': 'Obesity_I',
    'Obesidad II': 'Obesity_II',
    'Obesidad Extrema': 'Extreme_Obesity',
    'Desconocido': 'Unknown'
}
presentation_map = {
    'Cefálica': 1,
    'Podálica': 2,
    'Transversa': 3,
    'Otros': 9
}
plurality_map = {
    'Único': 1,
    'Gemelar': 2,
    'Trillizos': 3,
    'Orden Superior': 4
}
prenatal_map = {
    '1º-3º mes (1-12 semanas)': 1,
    '4º-6º mes (13-24 semanas)': 2,
    '7º mes o más (25+ semanas)': 3,
    'Sin control prenatal': 4,
    'Desconocido': 5
}

#### 3.2. Funciones de predicción

In [33]:
# Función de predicción de parto pretérmino
def predecir_parto_pretermino(
    age, race, education, birth_order, interval, prenatal_start, smoking,
    bmi, diabetes, hypertension, preterm, cesarean, no_risk, no_infect,
    chorio, presentation, plurality
):
    entrada = pd.DataFrame([{
        "Mother_Age": age_map[age],
        "Mother_Race": race_map[race],
        "Mother_Education": edu_map[education],
        "Total_Birth_Order": birth_order,
        "Interval_Last_Pregnancy": interval,
        "Prenatal_Care_Start": prenatal_map[prenatal_start],
        "Smoking_Before_Pregnancy": smoking,
        "BMI": bmi_map[bmi],
        "Gestational_Diabetes": binario(diabetes),
        "Gestational_Hypertension": binario(hypertension),
        "Previous_Preterm_Birth": binario(preterm),
        "Previous_Cesarean": binario(cesarean),
        "No_Risk_Factors": binario(no_risk),
        "No_Infections": binario(no_infect),
        "Chorioamnionitis": binario(chorio),
        "Fetal_Presentation": presentation_map[presentation],
        "Plurality": plurality_map[plurality]
    }])

    proba = modelo_xgb.predict_proba(entrada)[0][1]
    pred = modelo_xgb.predict(entrada)[0]
    nivel = "ALTO" if pred == 1 else "BAJO"
    emoji = "🔴" if pred == 1 else "🟢"
    texto = f"{emoji} Riesgo estimado de parto pretérmino: {proba:.1%} → {nivel}"

    porcentaje = round(proba * 100)
    return texto, porcentaje

# Función auxiliar para respuestas binarias
def binario(valor):
    return 1 if valor == "Sí" else 0

#### 3.3. Opciones de selección

In [34]:
age_opts = ['<15 años', '15-19 años', '20-24 años', '25-29 años', '30-34 años', '35-39 años', '40-44 años', '45-49 años', '50-54 años']
race_opts = ['Blanca', 'Negra', 'Indígena Americano/Nativo de Alaska', 'Asiática', 'Isleña del Pacífico', 'Multiracial']
edu_opts = ['Educación Primaria', 'Educación Secundaria', 'Bachillerato', 'Estudios Pre-Universitarios', 'Grado Superior', 'Graduado Universitario', 'Máster Universitario', 'Doctorado', 'Desconocido']
bmi_opts = ['Infrapeso', 'Normal', 'Sobrepeso', 'Obesidad I', 'Obesidad II', 'Obesidad Extrema', 'Desconocido']
pres_opts = ['Cefálica', 'Podálica', 'Transversa', 'Otros']
plural_opts = ['Único', 'Gemelar', 'Trillizos', 'Orden Superior']
birth_order_opts = list(range(1, 10))
prenatal_opts = list(prenatal_map.keys())

#### 3.4. Desarrollo de la interfaz

In [35]:
# Interfaz Gradio
with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown(
        "<h2 style='text-align:center; color:#2c3e50;'>Calculadora de Riesgo de Parto Pretérmino</h2>"
    )
    with gr.Row():
        age = gr.Dropdown(age_opts, label="Edad de la madre")
        race = gr.Dropdown(race_opts, label="Raza")
        education = gr.Dropdown(edu_opts, label="Nivel educativo")

    with gr.Row():
        birth_order = gr.Dropdown(birth_order_opts, label="Número de embarazos (contando actual)")
        interval = gr.Slider(0, 99, label="Meses desde el último embarazo", step=1)
        prenatal_start = gr.Dropdown(prenatal_opts, label="Inicio del cuidado prenatal (mes)")

    with gr.Row():
        smoking = gr.Slider(0, 20, label="Cigarrillos por día antes del embarazo", step=1)
        bmi = gr.Dropdown(bmi_opts, label="Índice de Masa Corporal (IMC)")
        presentation = gr.Dropdown(pres_opts, label="Presentación fetal")

    with gr.Row():
        plurality = gr.Dropdown(plural_opts, label="Número de fetos")
        diabetes = gr.Radio(["No", "Sí"], label="¿Diabetes gestacional?")
        hypertension = gr.Radio(["No", "Sí"], label="¿Hipertensión gestacional?")

    with gr.Row():
        preterm = gr.Radio(["No", "Sí"], label="¿Parto prematuro previo?")
        cesarean = gr.Radio(["No", "Sí"], label="¿Cesárea previa?")
        no_risk = gr.Radio(["No", "Sí"], label="¿Sin factores de riesgo?")

    with gr.Row():
        no_infect = gr.Radio(["No", "Sí"], label="¿Sin infecciones?")
        chorio = gr.Radio(["No", "Sí"], label="¿Corioamnionitis?")

    gr.Markdown("---")
    btn = gr.Button("🔎 Calcular riesgo", variant="primary")
    salida_texto = gr.Textbox(label="Resultado", lines=2, elem_classes="text-center")
    barra_riesgo = gr.Slider(0, 100, step=1, label="Visualización del riesgo (%)", interactive=False)

    btn.click(fn=predecir_parto_pretermino,
              inputs=[age, race, education, birth_order, interval, prenatal_start, smoking,
                      bmi, diabetes, hypertension, preterm, cesarean, no_risk, no_infect,
                      chorio, presentation, plurality],
              outputs=[salida_texto, barra_riesgo])

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93e3f59b415d49cae3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
